In [1]:
import h5py
import numpy as np
import os
import pickle
import tifffile
from tqdm import tqdm

In [23]:
def read_hdf5(fn):
    f = h5py.File(fn)
    k = ''
    for key in f.keys():
        k = key
    return f[key][()].astype('int')

def arr_to_cells(cells, rna_masks, roi, masks, markers):
    cell_id = []
    dict_l = []
    for i in range(len(masks)):
        dic = {}
        mask = (masks[i]>0).astype('int')
        temp = rna_masks * mask
        for j in range(temp.shape[2]):
            marker = markers[j]
            idx = np.where(temp[:,:,j])
            dic[marker] = list(zip(idx[0], idx[1]))
        _ = dic.pop('EMPTY')
        dict_l.append(dic)
        cell_id.append(cells+'_'+roi+"_"+str(i+1))
    return cell_id, dict_l

def save_pickles(out_dir, cell_id, dict_l):
    for i in range(len(cell_id)):
        fn = os.path.join(out_dir, cell_id[i])+'.pkl'
        pickle.dump(dict_l[i], open(fn, 'wb'))

def exp_from_dic(position_dic, gene_order):
    arr = []
    for i in range(len(gene_order)):
        g = gene_order[i]
        if g=='EMPTY':
            continue
        else:
            arr.append(len(position_dic[g]))
    return np.array(arr)

In [46]:
in_dir = r'..\images\HCH\registered'
out_dir = r'..\results\hch\detected_dots'
fn_l = os.listdir(in_dir)
fn_l.sort()
markers_l = []
l = os.listdir(os.path.join(in_dir, fn_l[0]))
for n in l:
    if n.startswith('cycle'):
        n = n[:len(n)-4]
        markers_l = markers_l + n.split('_')[1:4]
for i in range(len(markers_l)):
    markers_l[i] = markers_l[i].upper()
print(markers_l)

['EMPTY', 'EEF2', 'ACTB', 'SOX9', 'GAPDH', 'SPP1', 'IL8', 'IL6', 'CCL11', 'COL5A2', 'COL1A1', 'PDL1', 'MALAT1', 'RUNX1', 'CXCR4', 'EMPTY', 'MKI67', 'NANOG', 'CONA', 'PHA', 'WGA']


In [47]:
roi = '001'
cells = 'hch'
for roi in tqdm(fn_l):
    l = os.listdir(os.path.join(in_dir, roi))
    l.sort()
    masks = []
    for n in l:
        if n.startswith('cell'):
            mask = tifffile.imread(os.path.join(in_dir, roi, n))
            if len(mask.shape) == 3:
                mask = mask.reshape(mask.shape[1], mask.shape[2], 1)
            if len(mask.shape) == 2:
                mask = mask.reshape(mask.shape[0], mask.shape[1], 1)
            masks.append(mask)
    rna_mask = read_hdf5(os.path.join(in_dir, roi, 'gene_count_abs_threshold_'+roi+'.hdf5'))
    cell_id, dict_l = arr_to_cells(cells, rna_mask, roi, masks, markers_l)
    save_pickles(out_dir, cell_id, dict_l)

 91%|██████████████████████████████████████████████████████████████████████████▍       | 59/65 [04:28<00:27,  4.55s/it]


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'Y:\\coskun-lab\\Zhou\\4_HCR\\20210614_hchCulture\\00_registered_images\\new\060\gene_count_abs_threshold_060.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)